In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stomach_Cancer"
cohort = "GSE130823"

# Input paths
in_trait_dir = "../../input/GEO/Stomach_Cancer"
in_cohort_dir = "../../input/GEO/Stomach_Cancer/GSE130823"

# Output paths
out_data_file = "../../output/preprocess/Stomach_Cancer/GSE130823.csv"
out_gene_data_file = "../../output/preprocess/Stomach_Cancer/gene_data/GSE130823.csv"
out_clinical_data_file = "../../output/preprocess/Stomach_Cancer/clinical_data/GSE130823.csv"
json_path = "../../output/preprocess/Stomach_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE130823_family.soft.gz', 'GSE130823_series_matrix.txt.gz']
Identified SOFT files: ['GSE130823_family.soft.gz']
Identified matrix files: ['GSE130823_series_matrix.txt.gz']



Background Information:
!Series_title	"Dissecting Expression Profiles of Gastric Precancerous Lesions and Early Gastric Cancer to Explore Crucial Molecules in Intestinal-type Gastric Cancer Tumorigenesis"
!Series_summary	"To investigate the changes in molecular expression, biological processes, stemness, immune microenvironment, tumor hallmark activities and co-expression relationships during intestinal-type gastric cancer carcinogenesis and to excavate the prognostic information contained in the carcinogenesis process. RNA expression profiles of ninety-four gastroscope biopsy samples with different stages of precancerous lesions or early gastric cancers and their paired controls were detected by Agilent Microarray."
!Series_overall_design	"RNA expression profiles of ninety-four gastroscope biopsy samples with different stages of precancerous lesions or early gastric cancers and their paired controls were detected by Agilent Microarray."

Sample Characteristics Dictionary:
{0: ['tissu

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains RNA expression profiles 
# detected by Agilent Microarray, which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary:
# - trait: Not explicitly available as a separate category
# - age: Available at key 2
# - gender: Available at key 1

# Looking at the background information, this study focuses on gastric cancer,
# but we don't have a clear indicator of cancer status in the provided characteristics.
# Therefore, we cannot determine trait status at this stage.
trait_row = None  # Cannot determine cancer status from available sample characteristics
age_row = 2       # Age information is at key 2
gender_row = 1    # Gender information is at key 1

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert cancer status to binary trait values.
    Not implemented as we cannot determine trait status.
    """
    return None  # Cannot determine trait status

def convert_age(value):
    """Convert age string to numerical value."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return int(value)
    except:
        return None

def convert_gender(value):
    """Convert gender string to binary (0 for female, 1 for male)."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Since trait_row is None, set is_trait_available to False
is_trait_available = False

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Skip clinical feature extraction since trait data is not available (trait_row is None)
print("Clinical feature extraction skipped because trait data is not available.")


Clinical feature extraction skipped because trait data is not available.


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315459', 'A_19_P00315482', 'A_19_P00315492', 'A_19_P00315493',
       'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519'],
      dtype='object', name='ID')

Gene expression data shape: (50739, 94)


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers observed appear to be microarray probe IDs (like "(+)E1A_r60_1", "A_19_P00315452"), not standard human gene symbols.
# These look like Agilent microarray probe identifiers which need to be mapped to human gene symbols.

requires_gene_mapping = True

# Print the conclusion for clarity
print(f"Gene mapping required: {requires_gene_mapping}")
print("These appear to be Agilent microarray probe IDs, not standard human gene symbols.")


Gene mapping required: True
These appear to be Agilent microarray probe IDs, not standard human gene symbols.


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448', 'A_33_P3318220'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'GB_ACC': [nan, nan, 'NM_015987', 'NM_080671', 'NM_178466'], 'LOCUSLINK_ID': [nan, nan, 50865.0, 23704.0, 128861.0], 'GENE_SYMBOL': [nan, nan, 'HEBP1', 'KCNE4', 'BPIFA3'], 'GENE_NAME': [nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4', 'BPI fold containing family A, member 3'], 'UNIGENE_ID': [nan, nan, 'Hs.642618', 'Hs.348522', 'Hs.360989'], 'ENSEMBL_ID': [nan, nan, 'ENST00000014930', 'ENST00000281830', 'ENST00000375454'], 'ACCESSION_STRING': [nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788', 'ref|NM_178466|ens|ENST00000375454|ens|ENST00000471233|tc

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the data to determine appropriate columns for mapping
# From the gene annotation preview, we can see:
# - 'ID' matches the probe identifiers from the gene expression data (like "A_33_P3246448")
# - 'GENE_SYMBOL' contains the human gene symbols (like "HEBP1", "KCNE4")

# 2. Extract mapping between probe IDs and gene symbols
# Define the column names for the identifiers and gene symbols
probe_id_col = 'ID'
gene_symbol_col = 'GENE_SYMBOL'

# Get the mapping dataframe using the helper function
gene_mapping_df = get_gene_mapping(gene_annotation, prob_col=probe_id_col, gene_col=gene_symbol_col)

# Print mapping info
print(f"Gene mapping dataframe shape: {gene_mapping_df.shape}")
print("First few rows of gene mapping:")
print(gene_mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)

# Normalize gene symbols to handle synonyms and case variations
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print info about the resulting gene expression data
print(f"\nResulting gene expression data shape: {gene_data.shape}")
print("First few rows of gene expression data:")
print(gene_data.head())


Gene mapping dataframe shape: (46204, 2)
First few rows of gene mapping:
              ID          Gene
2   A_23_P117082         HEBP1
3  A_33_P3246448         KCNE4
4  A_33_P3318220        BPIFA3
5  A_33_P3236322  LOC100129869
6  A_33_P3319925          IRG1



Resulting gene expression data shape: (19847, 94)
First few rows of gene expression data:
          GSM3754774  GSM3754775  GSM3754776  GSM3754777  GSM3754778  \
Gene                                                                   
A1BG       -0.156597   -0.789875    1.512063    0.087390    0.577830   
A1BG-AS1   -0.055273   -1.115357    1.203011    0.361480    0.520860   
A1CF        0.060761    4.167142    0.721337    3.379362    3.042676   
A2M         1.730769   -0.925951    1.529119    0.470597   -0.023590   
A2ML1       0.114604   -0.132678   -0.101793    0.342719    0.013418   

          GSM3754779  GSM3754780  GSM3754781  GSM3754782  GSM3754783  ...  \
Gene                                                                  ...   
A1BG       -0.408904    0.921989   -1.117775   -1.007375   -0.585641  ...   
A1BG-AS1   -0.628904    0.711314   -0.730740   -0.651767   -1.438113  ...   
A1CF        0.437447    0.190483    1.911350    2.402059    4.377823  ...   
A2M        -0.87222

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = gene_data  # It's already normalized in Step 6
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Check if trait data is available (from Step 2 we determined it was not)
is_trait_available = False
print(f"Trait data availability: {'Available' if is_trait_available else 'Not available'}")

# Since trait data is not available, we cannot create clinical features or linked data
# We'll use the initial validation since we can't perform the final validation without trait data
validate_result = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=is_trait_available
)

print(f"Gene expression data was processed and saved, but no linked data was created due to missing trait information.")

Normalized gene data shape: (19847, 94)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06', 'AAA1']


Normalized gene data saved to ../../output/preprocess/Stomach_Cancer/gene_data/GSE130823.csv
Trait data availability: Not available
Gene expression data was processed and saved, but no linked data was created due to missing trait information.
